### Import necessary Libraries

In [91]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [92]:
CLIENT_ID = 'JWCXQV2NXFPF34JEC0NO5CCOILZAVOLRKIDDKYABUG2YZTXS' # your Foursquare ID
CLIENT_SECRET = 'N0B1Z43N0LXX3TRYSTUVAPUQRJPILS420BTEDUDJUTLPEV5U' # your Foursquare Secret
VERSION = '20200619'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JWCXQV2NXFPF34JEC0NO5CCOILZAVOLRKIDDKYABUG2YZTXS
CLIENT_SECRET:N0B1Z43N0LXX3TRYSTUVAPUQRJPILS420BTEDUDJUTLPEV5U


#### let's start by converting the center of Yuen Long location to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [93]:
address = 'Yuen Long, HK'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

22.4426463 114.0304342


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Let's define a query to search for commercial properties that is within 1000 metres from the center of Yuen Long. 

In [94]:
radius = 1000 
LIMIT = 300
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
   longitude, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [95]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.neighborhood', 'venue.venuePage.id'],
      dtype='object')

####  To create the map of nearby venues in Yuen Long

In [96]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [97]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head(100)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Aya (彩),"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",22.441018,114.030366
1,Ichininmae (一人前),"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",22.442218,114.031530
2,亞玉豆腐花,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",22.444035,114.031049
3,發記腸粉粥品,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",22.442320,114.031150
4,過橋麵檔,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",22.441612,114.030452
...,...,...,...,...
73,Port Spot Café 葡點,"[{'id': '4def73e84765ae376e57713a', 'name': 'P...",22.442144,114.022052
74,Paper Stone Bakery,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",22.445164,114.038386
75,Fantastic Café (悠閑閣),"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",22.436577,114.024651
76,Glass House,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",22.445443,114.038846


In [98]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(100)

,name,categories,lat,lng
0,Aya (彩),Ramen Restaurant,22.441018,114.030366
1,Ichininmae (一人前),Noodle House,22.442218,114.031530
2,亞玉豆腐花,Dessert Shop,22.444035,114.031049
3,發記腸粉粥品,Chinese Restaurant,22.442320,114.031150
4,過橋麵檔,Chinese Restaurant,22.441612,114.030452
...,...,...,...,...
73,Port Spot Café 葡點,Portuguese Restaurant,22.442144,114.022052
74,Paper Stone Bakery,Bakery,22.445164,114.038386
75,Fantastic Café (悠閑閣),Café,22.436577,114.024651
76,Glass House,Restaurant,22.445443,114.038846


In [99]:
# Top 10Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]


Chinese Restaurant      11
Noodle House             7
Fast Food Restaurant     5
Café                     4
Thai Restaurant          3
Coffee Shop              3
Japanese Restaurant      3
Shopping Mall            3
Dessert Shop             3
Sushi Restaurant         2
Name: categories, dtype: int64

In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
# Nearby Venues
YL_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

Aya (彩)
Ichininmae (一人前)
亞玉豆腐花
發記腸粉粥品
過橋麵檔
HABITŪ caffè
Man Fong Café (萬芳冰室)
Hang Heung (恆香老餅家)
Joint Publishing (三聯書店)
Old Fung Tea House (老馮茶居)
Tai Wing Wah Restaurant (大榮華酒樓)
Sushi Man (鮨文)
Starbucks (星巴克)
Wing Nin Noodle (永年士多)
士林台灣麵
My Cinema
嘉麗園
Tai Kiu Market (大橋街市)
Accro Coffee
simplylife BAKERY CAFÉ
KFC (肯德基)
Shaffi's Indian Restaurant
PizzaExpress
Kei Kee Dessert (佳記甜品)
寿司の神
NEW YORK Café n Bar
Starbucks
Ho To Tai Noodle Shop (Ho To Tai Noodle Shop 好到底麵家)
McDonald's (麥當勞)
Tung Yick Market (同益街市)
Tin Hung Restaurant (天鴻燒鵝飯店)
petit An Nam (安南小館)
Yoho Mall (形點)
McDonald's
Ahmad Curry House (亞曼咖哩屋)
Yuen Long Theatre (元朗劇院)
McDonald's (麥當勞)
MTR Yuen Long Station (港鐵元朗站)
Kolour Yuen Long (元朗千色匯)
Tanaka Japanese Restaurant (田中壽司刺身店)
Modern Shanghai (家上海)
Yuen Long Plaza (元朗廣場)
Victory Beef Ball 勝利牛丸
佳記甜品（B仔涼粉）
新記煲仔雞粥
Taste
Plentiful Delight Banquet Restaurant (喜尚嘉喜宴會廳)
雲貴軒
TamJai SamGor Mixian 譚仔三哥米線
Mint (薄荷葉泰國菜)
Fulum Palace (富臨皇宮)
Yuen Long Theatre (元朗戲院)
McDonald's (麥當勞)
Tomato 

In [102]:
YL_venues.groupby('Neighborhood').count().head(100)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ANA Sushi (穴壽司),39,39,39,39,39,39
Accro Coffee,30,30,30,30,30,30
Ahmad Curry House (亞曼咖哩屋),25,25,25,25,25,25
Aya (彩),52,52,52,52,52,52
Azores (亞蘇爾),32,32,32,32,32,32
...,...,...,...,...,...,...
怡怡泰菜,25,25,25,25,25,25
新記煲仔雞粥,49,49,49,49,49,49
發記腸粉粥品,51,51,51,51,51,51


In [103]:
f=YL_venues.groupby('Neighborhood').count().head(100)
nei_most_venue=f.sort_values('Venue',ascending=False)
nei_most_venue.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
McDonald's (麥當勞),149,149,149,149,149,149
Tanaka Japanese Restaurant (田中壽司刺身店),62,62,62,62,62,62
NEW YORK Café n Bar,61,61,61,61,61,61
Kei Kee Dessert (佳記甜品),60,60,60,60,60,60
Fulum Palace (富臨皇宮),59,59,59,59,59,59


In [104]:
map_YL = folium.Map(location=[latitude, longitude], zoom_start=15)
 
for lat, lng in zip(nearby_venues['lat'], nearby_venues['lng']):
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_YL)  
map_YL

#### Therefore, we can see the top 5 neighborhoods with most venues are: McDonald's (麥當勞),Tanaka Japanese Restaurant (田中壽司刺身店),NEW YORK Café n Bar,Kei Kee Dessert (佳記甜品),Fulum Palace (富臨皇宮)	

In [111]:
nearby_venues.loc[nearby_venues['name'] == "McDonald's (麥當勞)"]

,name,categories,lat,lng
28,McDonald's (麥當勞),Fast Food Restaurant,22.444466,114.028651
36,McDonald's (麥當勞),Fast Food Restaurant,22.445382,114.034374
52,McDonald's (麥當勞),Fast Food Restaurant,22.444577,114.024047


In [112]:
nearby_venues.loc[nearby_venues['name'] == "Tanaka Japanese Restaurant (田中壽司刺身店)"]


,name,categories,lat,lng
39,Tanaka Japanese Restaurant (田中壽司刺身店),Sushi Restaurant,22.444476,114.031615


In [113]:
nearby_venues.loc[nearby_venues['name'] == "NEW YORK Café n Bar"]


,name,categories,lat,lng
25,NEW YORK Café n Bar,American Restaurant,22.443848,114.032416


In [114]:
nearby_venues.loc[nearby_venues['name'] == "Kei Kee Dessert (佳記甜品)"]


,name,categories,lat,lng
23,Kei Kee Dessert (佳記甜品),Dessert Shop,22.443157,114.03349


In [115]:
nearby_venues.loc[nearby_venues['name'] == "Fulum Palace (富臨皇宮)"]

,name,categories,lat,lng
50,Fulum Palace (富臨皇宮),Chinese Restaurant,22.443264,114.033931


In [117]:
map_best_loc = folium.Map(location=[latitude, longitude], zoom_start=15)


folium.Marker(
    location=[22.444466, 114.028651],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(map_best_loc)

folium.Marker(
    location=[22.444476, 114.031615],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(map_best_loc)

folium.Marker(
    location=[22.443848, 114.032416],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(map_best_loc)

folium.Marker(
    location=[22.443157, 114.03349],
    popup='Some Other Location',
    icon=folium.Icon(color='yellow', icon='info-sign')
).add_to(map_best_loc)

folium.Marker(
    location=[22.443264, 114.033931],
    popup='Some Other Location',
    icon=folium.Icon(color='black', icon='info-sign')
).add_to(map_best_loc)

map_best_loc


#### **The blue/cloud color** is the location I recommend to build the second branch church. It happens that the area is exactly the same as the church I used to go to.